# Import library

In [243]:
import requests
import json
import pandas as pd
import numpy as np

In [244]:
url_fcp_swagger = "https://fr1pslcmf05:8770/api/"
url_pricing_config = "https://fr1pslcmf05:8770/api/pricing/configs"
url_result_handler ="https://fr1pslcmf05:8770/api/pricing/report/result/Pricing"
url_measureGRoup = "https://fr1pslcmf05:8770/api/pricing/measureGroups"

# Mandatory field : we use those defined in FCP no need to define it

In [245]:
configName="DEFAULT"
pricingMethod="THEORETICAL"
marketDataSetId="$id/DEFAULT"
marketDataProviderId="PE_STORE_MDP"
resultHandlerConfigId=None
resultHandlerId="Collector"

# Helper function for REST call : GET / PUT / POST /DELETE

In [246]:
#helper
def get(url):
    """REST CALL : GET"""
    reponse = requests.get(url, verify=False)
    if (reponse.ok):
        val = reponse.json()
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def delete(url ):
    """REST CALL : DELETE"""
    reponse = requests.delete(url,verify=False)
    if (reponse.ok):
        val = "data deleted"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def put(url, json):
    """REST CALL : PUT"""
    reponse = requests.put(url, json=json, verify=False)
    if (reponse.ok):
        val = "build"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val

def post(url, json):
    """REST CALL : POST"""
    reponse = requests.post(url, json=json, verify=False,headers={'Connection':'close'})
    if(reponse.ok):
        if(reponse.status_code==201):
            val = reponse.headers._store['location'][1]
        else:
            val = reponse.json()
    elif(reponse.status_code==400):
        val= reponse.json()
    else:
        status = reponse.raise_for_status()
        val =  "error : " + status
    reponse.close()
    return val



In [247]:
def getPricingId(configName):
    urlConfig = "https://fr1pslcmf05:8770/api/pricing/configs"
    config = get(urlConfig)
    for item in config:
        if (item['name']==configName):
            return item["id"]
    return "error"

In [248]:
def unitary(aod,referenceCurrency,scenarioContexts,perimeter,configName =configName, pricingMethod = pricingMethod,marketDataSetId = marketDataSetId,marketDataProviderId=marketDataProviderId,resultHandlerId = resultHandlerId,resultHandlerConfigId=resultHandlerConfigId):
    unitary_price = {}
    unitary_price["asOfDate"] = aod
    unitary_price["referenceCurrency"] = referenceCurrency     
    unitary_price["pricingMethod"] = pricingMethod
    unitary_price["marketDataSetId"] = marketDataSetId
    unitary_price["resultHandlerId"] = resultHandlerId
    unitary_price["resultHandlerConfigId"] = resultHandlerConfigId
    unitary_price["marketDataProviderId"] = marketDataProviderId    
    unitary_price["pricingConfigId"] = getPricingId(configName)
    unitary_price["perimeter"] = perimeter
    unitary_price["scenarioContexts"] = scenarioContexts
    url_price = url_fcp_swagger + "pricing/price/"
    reponse = post(url=url_price, json=unitary_price)
    return reponse["pricingResults"]


# NPV use case

In [249]:
#input needed : that need to be defined
scenarioContexts =[
    {
      "id": "base",
      "measureGroupIds": [
        "NPV"
      ]
    }
]
perimeter = {"trade" : {"ids": ["FXOptionVanilla_1"]}}

In [250]:
resultJson = unitary(aod="2016-07-04",referenceCurrency="$id/USD",perimeter=perimeter, scenarioContexts = scenarioContexts)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [251]:
resultJson

{'FXOptionVanilla_1': {'currencyId': '$id/USD',
  'pricerId': {'library': 'FQPFXDerivative',
   'pricingModel': 'Black Scholes',
   'numericalMethod': 'Closed Formula'},
  'scenarios': [{'id': 'base',
    'entries': [{'id': '0',
      'measures': {'NPV': -6.847514388329348e-13},
      'properties': {},
      'cashFlows': [],
      'legs': []}]}]}}

   # create a measureGRoup

In [252]:
measureGroupTD = {
    "id" : "FXO_TC_ID", 
    "name": "FXO_TRADE_CAPTURE",
     "measureIds": [
         "NPV",
         "FXD_Delta_Forward",
         "FXD_Delta_Spot",
         "FXD_Gamma",
         "FXD_Theta",
         "FXD_Rho_Domestic",
         "FXD_Rho_Foreign",
         "FXD_Vega",         
     ]
}

In [253]:
val = post(url_measureGRoup,measureGroupTD)
if("status" in val.keys()):
    print(val["detail"])
else:
    id_measure_group = val["id"]


C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [254]:
get(url_measureGRoup+"/"+id_measure_group)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': '78b6a8220e4340f59a883d7a9d870218',
 'name': 'FXO_TRADE_CAPTURE',
 'measureIds': ['FXD_Delta_Forward',
  'FXD_Delta_Spot',
  'FXD_Gamma',
  'FXD_Rho_Domestic',
  'FXD_Rho_Foreign',
  'FXD_Theta',
  'FXD_Vega',
  'NPV']}

In [255]:
#input needed : that need to be defined
scenarioContexts =[
    {
      "id": "base",
      "measureGroupIds": [
        id_measure_group
      ]
    }
]
perimeter = {"trade" : {"ids": ["FXOptionVanilla_1"]}}

In [239]:
resultJson = unitary(aod="2016-07-04",perimeter=perimeter, scenarioContexts = scenarioContexts)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [240]:
resultJson

{'FXOptionVanilla_1': {'currencyId': '$id/USD',
  'pricerId': {'library': 'FQPFXDerivative',
   'pricingModel': 'Black Scholes',
   'numericalMethod': 'Closed Formula'},
  'scenarios': [{'id': 'base',
    'entries': [{'id': '0',
      'measures': {'FXD_Gamma': -1.2887125171278262e-07,
       'FXD_Theta': -4.095281443522673e-08,
       'NPV': -6.847514388329348e-13,
       'FXD_Vega': -5.614785707219413e-10,
       'FXD_Delta_Forward': -2.992289957233029e-10,
       'FXD_Rho_Foreign': 1.829230563753891e-11,
       'FXD_Delta_Spot': -2.992958381612741e-10,
       'FXD_Rho_Domestic': 9.381223815569829e-07},
      'properties': {},
      'cashFlows': [],
      'legs': []}]}]}}

In [241]:
delete(url_measureGRoup+"/"+id_measure_group)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'data deleted'

In [242]:
get(url_measureGRoup)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'id': 'NPV', 'name': 'NPV', 'readOnly': True},
 {'id': 'Standard', 'name': 'Standard', 'readOnly': True},
 {'id': 'Advanced', 'name': 'Advanced', 'readOnly': True}]